In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from fvcore.nn import FlopCountAnalysis

import optuna

In [9]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
DIR = ".."
BATCHSIZE = 128
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10
INPUT_SIZE = 28 * 28

def generate_model(n_layers, n_units):
    layers = []

    in_features = INPUT_SIZE
    for i in range(n_layers):
        out_features = n_units[i]

        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(0.25))

        in_features = out_features

    layers.append(nn.Linear(in_features, 10))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)


# Defines training and evaluation.
def train_model(model, optimizer, train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.view(-1, INPUT_SIZE).to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        F.nll_loss(model(data), target).backward()
        optimizer.step()


def eval_model(model, valid_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(valid_loader):
            data, target = data.view(-1, INPUT_SIZE).to(DEVICE), target.to(DEVICE)
            pred = model(data).argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = correct / N_VALID_EXAMPLES

    flops = FlopCountAnalysis(model, inputs=(torch.randn(1, INPUT_SIZE).to(DEVICE),)).total()
    return flops, accuracy

In [10]:
def max_objective(lr, n_layers, n_units):
    train_dataset = torchvision.datasets.FashionMNIST(
        DIR, train=True, download=True, transform=torchvision.transforms.ToTensor()
    )
    train_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(train_dataset, list(range(N_TRAIN_EXAMPLES))),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    val_dataset = torchvision.datasets.FashionMNIST(
        DIR, train=False, transform=torchvision.transforms.ToTensor()
    )
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(val_dataset, list(range(N_VALID_EXAMPLES))),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    model = generate_model(n_layers, n_units).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr)

    for epoch in range(2):
        train_model(model, optimizer, train_loader)
    flops, accuracy = eval_model(model, val_loader)
    return -flops, accuracy

In [11]:
def optuna_objective(trial):
    n_layers = trial.suggest_int("n_layers", 1, 3)
    n_units = []
    for i in range(n_layers):
        n_units.append(trial.suggest_int("n_units_l{}".format(i), 16, 64))
    lr = trial.suggest_int("lr", 1, 5)
    lr = 10**(-lr)

    return max_objective(lr, n_layers, n_units)

In [12]:
study = optuna.create_study(directions=["maximize", "maximize"])
study.optimize(optuna_objective, n_trials=10, timeout=120)

print("Number of finished trials: ", len(study.trials))

[I 2025-02-07 17:40:34,761] A new study created in memory with name: no-name-4a0f73ca-e885-453d-a4cc-f5fa3acf560a
Unsupported operator aten::log_softmax encountered 1 time(s)
[I 2025-02-07 17:40:39,879] Trial 0 finished with values: [-43302.0, 0.15703125] and parameters: {'n_layers': 3, 'n_units_l0': 49, 'n_units_l1': 54, 'n_units_l2': 35, 'lr': 5}.
Unsupported operator aten::log_softmax encountered 1 time(s)
[I 2025-02-07 17:40:42,737] Trial 1 finished with values: [-44464.0, 0.353125] and parameters: {'n_layers': 1, 'n_units_l0': 56, 'lr': 1}.
Unsupported operator aten::log_softmax encountered 1 time(s)
[I 2025-02-07 17:40:46,341] Trial 2 finished with values: [-16596.0, 0.41796875] and parameters: {'n_layers': 3, 'n_units_l0': 19, 'n_units_l1': 44, 'n_units_l2': 16, 'lr': 3}.
Unsupported operator aten::log_softmax encountered 1 time(s)
[I 2025-02-07 17:40:50,979] Trial 3 finished with values: [-42082.0, 0.76015625] and parameters: {'n_layers': 1, 'n_units_l0': 53, 'lr': 2}.
Unsuppor

Number of finished trials:  10


In [ ]:
from vopy.maximization_problem import Problem

